In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import seaborn as sns
py.init_notebook_mode(connected=True)
%matplotlib inline

Using TensorFlow backend.


In [2]:
data = pd.read_csv(filepath_or_buffer="./Data/clean_data_new.csv", index_col="date")

In [3]:
data = data.apply(pd.to_numeric, errors = "coerce")
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7931 entries, 19870520 to 20181102
Data columns (total 8 columns):
TCMNOM_Y10          7931 non-null float64
TCMNOM_Y2           7931 non-null float64
DCOILBRENTEU        7931 non-null float64
spindx              7931 non-null float64
GOLDPMGBD228NLBM    7931 non-null float64
exalus              7931 non-null float64
exjpus              7931 non-null float64
exukus              7931 non-null float64
dtypes: float64(8)
memory usage: 557.6 KB


In [4]:
data['spindx'].replace(0, np.nan, inplace=True)
data['spindx'].fillna(method='ffill', inplace=True)

In [5]:
#reshaping prediction value into value range -1 to 1
from sklearn.preprocessing import MinMaxScaler

In [6]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    #print(cols)
#     # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# Data Transformation

In [69]:
number_of_variables = 8
values = data[['spindx'] + ['TCMNOM_Y2'] + ['TCMNOM_Y10'] + ['DCOILBRENTEU'] + ['GOLDPMGBD228NLBM'] + ['exalus'] + ['exjpus'] + ['exukus']].values
values = values.astype('float32')

In [70]:
#getting the last 3 days' data
historic_data = np.array([])
for day in values[-3:]:
    #historic_data = np.concatenate((values[-3], values[-2], values[-1]), axis=None)
    historic_data = np.concatenate((historic_data, day), axis=None)

In [71]:
historic_data[:8]

array([2.71173999e+03, 2.86999989e+00, 3.15000010e+00, 7.48399963e+01,
       1.21494995e+03, 1.41040003e+00, 1.12860001e+02, 7.82500029e-01])

In [72]:
historic_data = np.append (historic_data, historic_data[:8]) #add a dummy for prediction

In [73]:
#historic_data

In [74]:
#convert data to become supervised 
look_back = 3 
time_steps = 1
#series_to_supervised(values, look_back, time_steps)
reframed = series_to_supervised(values, look_back, time_steps)

In [76]:
#look_back = 5
number_of_variables = 8
#keeping first varible in first period
# reframed.drop(reframed.columns[-1 * number_of_variables + 1:], axis=1, inplace=True) 
# reframed.drop(reframed.columns[look_back*number_of_variables:-1], axis=1, inplace=True)

In [77]:
reframed = reframed.append(dict(zip(reframed.columns, historic_data)), ignore_index=True)
#reframed

In [78]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_pred = scaler.fit_transform(reframed)

In [79]:
scaled_pred.shape

(7929, 32)

In [80]:
pred_para = scaled_pred[-1][:-8]

In [81]:
pred_para = pred_para.reshape(1,3,8)

In [82]:
pred_para.shape

(1, 3, 8)

# Load Market and predict

In [83]:
#plot prediction
from keras.models import load_model

multi_model = load_model("market_lstm_modelv3_weights.hdf5")
yhat = multi_model.predict(pred_para)

In [84]:
yhat

array([[0.8680495 , 0.28463444, 0.17747194, 0.5117416 , 0.61518323,
        0.40173227, 0.43516776, 0.8245775 ]], dtype=float32)

In [85]:
pred_para.shape

(1, 3, 8)

In [86]:
pred_para = pred_para.reshape((1,24))
pred_para.shape

(1, 24)

In [87]:
pred_para

array([[0.91908985, 0.27766392, 0.20090296, 0.48750462, 0.58589085,
        0.43241157, 0.44119747, 0.87848608, 0.92966685, 0.27459015,
        0.19977429, 0.46088247, 0.59575571, 0.41575062, 0.44036592,
        0.8451906 , 0.92327189, 0.2817623 , 0.20880363, 0.45984428,
        0.59633417, 0.41411882, 0.44392966, 0.84661354]])

In [88]:
pred = concatenate((pred_para[:, :], yhat), axis=1)
pred

array([[0.91908985, 0.27766392, 0.20090296, 0.48750462, 0.58589085,
        0.43241157, 0.44119747, 0.87848608, 0.92966685, 0.27459015,
        0.19977429, 0.46088247, 0.59575571, 0.41575062, 0.44036592,
        0.8451906 , 0.92327189, 0.2817623 , 0.20880363, 0.45984428,
        0.59633417, 0.41411882, 0.44392966, 0.84661354, 0.8680495 ,
        0.28463444, 0.17747194, 0.51174158, 0.61518323, 0.40173227,
        0.43516776, 0.82457751]])

In [89]:
inv_pred = scaler.inverse_transform(pred)

In [91]:
inv_pred[:,-8:]

array([[2.57358243e+03, 2.93803216e+00, 2.94240127e+00, 7.81083505e+01,
        1.26305391e+03, 1.37467706e+00, 1.12352420e+02, 7.63556555e-01]])

In [92]:
inv_pred = inv_pred[:,-8:]

In [95]:
inv_pred

array([[2.57358243e+03, 2.93803216e+00, 2.94240127e+00, 7.81083505e+01,
        1.26305391e+03, 1.37467706e+00, 1.12352420e+02, 7.63556555e-01]])